In [ ]:
#Example NPZ FILE
!mkdir -p data/PittsburghBridge
!wget -P data/PittsburghBridge https://dl.fbaipublicfiles.com/pytorch3d/data/PittsburghBridge/pointcloud.npz

In [ ]:
import sys
import torch

need_pytorch3d = False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d = True
if need_pytorch3d:
    pyt_version_str = torch.__version__.split("+")[0].replace(".", "")
    version_str = "".join([
        f"py3{sys.version_info.minor}_cu",
        torch.version.cuda.replace(".", ""),
        f"_pyt{pyt_version_str}"
    ])
    !pip install iopath
    if sys.platform.startswith("linux"):
        print("Trying to install wheel for PyTorch3D")
        !pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
        pip_list = !pip freeze
        need_pytorch3d = not any(i.startswith("pytorch3d==") for i in pip_list)
    if need_pytorch3d:
        print(f"failed to find/install wheel for {version_str}")
if need_pytorch3d:
    print("Installing PyTorch3D from source")
    !pip install ninja
    !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

In [ ]:

from pytorch3d.structures import Pointclouds

def bounding_sphere_normalize(points: torch.Tensor) -> torch.Tensor:
    """
    points: (N,3) tensor of point coords
    Return normalized points in a unit sphere centered at origin.
    """
    center = points.mean(dim=0, keepdim=True)
    max_dist = (points - center).norm(p=2, dim=1).max()
    points_normed = (points - center) / max_dist
    return points_normed


def load_3d_data(file_path, num_points=10000, device="cuda", do_normalize=True):
    # Load NPZ point cloud directly like in the example
    pointcloud = np.load(file_path)
    verts = torch.Tensor(pointcloud['points']).to(device)

    # Check if 'colors' key exists, if not, create a default color
    if 'colors' in pointcloud:
        rgb = torch.Tensor(pointcloud['colors']).to(device)
    else:
        rgb = torch.ones(verts.shape[0], 3, device=device)  # Default to white color

    # Subsample if needed
    if len(verts) > num_points:
        idx = torch.randperm(len(verts))[:num_points]
        verts = verts[idx]
        rgb = rgb[idx]

    if do_normalize:
        verts = bounding_sphere_normalize(verts)

    # Return both the points tensor and the Pointclouds object
    point_cloud = Pointclouds(points=[verts], features=[rgb])
    return point_cloud  # Return both

In [ ]:
from itertools import islice

import torch
from pytorch3d.structures import Pointclouds
from pytorch3d.renderer import (
    look_at_view_transform,
    FoVOrthographicCameras,
    FoVPerspectiveCameras,
    PointsRasterizationSettings,
    PointsRenderer,
    PointsRasterizer,
    AlphaCompositor
)
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import torchvision.transforms as T


class MultiViewPointCloudRenderer:
    def __init__(self, image_size=512, base_dist=20, base_elev=10, base_azim=0,
                 device=torch.device("cuda" if torch.cuda.is_available() else "cpu")):
        self.device = device
        self.image_size = image_size
        self.base_dist = base_dist
        self.base_elev = base_elev
        self.base_azim = base_azim
        self.to_tensor = T.Compose([
            T.Resize((image_size, image_size)),
            T.ToTensor()
        ])

        # Define the settings for rasterization
        self.raster_settings = PointsRasterizationSettings(
            image_size=image_size,
            radius=0.008,
            points_per_pixel=20
        )

        # Define all views relative to base view
        self.views = {
            'Default': (base_dist, base_elev, base_azim),
            'Y_90deg': (base_dist, base_elev, base_azim + 90),
            'Y_180deg': (base_dist, base_elev, base_azim + 180),
            'Y_-90deg': (base_dist, base_elev, base_azim - 90),
            'X_90deg': (base_dist, base_elev + 90, base_azim),
            'X_-90deg': (base_dist, base_elev - 90, base_azim),
                    }


    def get_center_point(self, point_cloud):
        """Calculate the center point of the point cloud"""
        points = point_cloud.points_packed()
        center = torch.mean(points, dim=0)
        return center.unsqueeze(0)  # Add batch dimension

    def create_renderer(self, dist, elev, azim, center_point, background_color=(0, 0, 0)):
        """Create a renderer for specific camera parameters"""
        # Use the center point as the 'at' parameter
        R, T = look_at_view_transform(
            dist=dist,
            elev=elev,
            azim=azim,
            at=center_point,  # Look at the center of the point cloud
        )
        cameras = FoVPerspectiveCameras(
        device=self.device,
        R=R,
        T=T
        )

        rasterizer = PointsRasterizer(cameras=cameras, raster_settings=self.raster_settings)
        renderer = PointsRenderer(
            rasterizer=rasterizer,
            compositor=AlphaCompositor(background_color=background_color)
        )
        return renderer

    def load_background(self, background_path):
        bg_image = Image.open(background_path)
        bg_tensor = self.to_tensor(bg_image).to(self.device)
        return bg_tensor.permute(1, 2, 0)  # Convert to HWC format

    def render_all_views(self, point_cloud, n_views=6, background_path=None,background_color=(0, 0, 0)):
        images = {}
        center_point = self.get_center_point(point_cloud)

        if background_path: background = self.load_background(background_path)
        else:
            background = None

        for view_name, (dist, elev, azim) in islice(self.views.items(), n_views):
            renderer = self.create_renderer(dist, elev, azim, center_point,background_color=background_color)
            image = renderer(point_cloud)

            if background is not None:
                # Create binary mask from points
                mask = torch.any(image[0, ..., :3] > 0, dim=-1).float()
                mask = mask.unsqueeze(-1).expand(-1, -1, 3)
                composite = (image[0, ..., :3] * mask) + (background * (1 - mask))
                images[view_name] = composite
            else:
                images[view_name] = image[0, ..., :3]

        return images

In [ ]:
import os
import numpy as np
import torch
import torchvision

def save_results(point_cloud, renderer,n_views,device,output_dir,output_name):

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    rendered_images = renderer.render_all_views(point_cloud=point_cloud, n_views=n_views,background_color = (1,1,1))
    # Convert dictionary of images to tensor
    rendered_tensor = []
    for name, img in rendered_images.items():
        rendered_tensor.append(img.to(device))
    rendered_tensor = torch.stack(rendered_tensor)

    # Convert rendered images to CLIP format
    rendered_images = rendered_tensor.permute(0, 3, 1, 2)  # [B, H, W, C] -> [B, C, H, W]

    # Convert to uint8 range [0, 255]
    rendered_images = (rendered_images * 255).clamp(0, 255).to(torch.uint8)

    # Save rendered image using torchvision
    torchvision.utils.save_image(
        rendered_images.float() / 255.0,  # Convert back to [0,1] range
        os.path.join(output_dir, output_name),
         normalize=False  # We've already normalized the values
     )

In [ ]:
device="cuda"

point_cloud = load_3d_data(
    "/content/highlighted_points.npz",
    num_points=100000
)


renderer = MultiViewPointCloudRenderer(
    image_size=1024,
    base_dist=2.5,  # Your default view distance
    base_elev=10,  # Your default elevation
    base_azim=0,  # Your default azimuth
    device=device
)

save_results(
    point_cloud=point_cloud,
    renderer=renderer,
    n_views=6,
    output_dir="./output",
    output_name="point_cloud2.png",
    device=device
)

In [ ]:
from pytorch3d.vis.plotly_vis import plot_scene
plot_scene({
    "Pointcloud": {
        "person": point_cloud
    }
})

In [ ]:
import open3d as o3d
import numpy as np

# Load the .ply file
pcd = o3d.io.read_point_cloud("/content/gt_pointcloud_contain.ply")

# Extract points and colors
points = np.asarray(pcd.points)
colors = np.asarray(pcd.colors)

# Lighten the base colors (adjust factor as needed)
colors = colors * 1.5  # Increase brightness by 50%
colors = np.clip(colors, 0, 1)  # Clip values to the valid range [0, 1]

# Save to .npz
np.savez_compressed("output.npz", points=points, colors=colors)

Ground truth was saved as ply file so we convert npz

In [ ]:
!pip install open3d numpy

calculation of IOU

In [ ]:
!git clone https://github.com/amiralichangizi/Affordance3DHighlighter.git


In [ ]:
!pip install gdown
!gdown --id 1siZtGusB1LfQVapTvNOiYi8aeKKAgcDF
!unzip full-shape.zip -d /content/Affordance3DHighlighter/data/

In [ ]:
import os
os.chdir('/content/Affordance3DHighlighter')

Find the previously used shape in the dataset for aiou calculation

In [ ]:
import torch
from torch.utils.data import DataLoader
from src.data_loader_fullshape import FullShapeDataset, create_dataset_splits

# Load the dataset
pkl_path = "/content/Affordance3DHighlighter/data/full_shape_train_data.pkl"
device = "cpu"  # or "cuda"
# Instantiate the dataset
dataset = FullShapeDataset(pkl_path, device=device, target_classes=['Door'], target_affordances = ['openable', 'pushable', 'pull'])
train_data, val_data, test_data = create_dataset_splits(dataset, val_ratio=0.1, test_ratio=0.1)

# Print out your current dataset configuration
print("Current dataset target classes:", dataset.target_classes)
print("Current dataset target affordances:", dataset.target_affordances)

# Let's see what shapes we actually have
print("\nSample of available shape IDs:")
for i in range(min(5, len(dataset))):
    entry = dataset[i]
    print(f"Shape ID: {entry['shape_id']}")
    print(f"Class: {entry['shape_class']}")
    print(f"Affordances: {entry['affordances']}")
    print("---")

def find_shape_in_all_splits(shape_id, train_data, val_data, test_data):
    """
    Search for a specific shape ID across all dataset splits.
    Returns the shape entry and which split it was found in.
    """
    # Check training set
    for idx in range(len(train_data)):
        entry = train_data[idx]
        if entry['shape_id'] == shape_id:
            return entry, 'train', idx

    # Check validation set
    for idx in range(len(val_data)):
        entry = val_data[idx]
        if entry['shape_id'] == shape_id:
            return entry, 'split', idx

    # Check test set
    for idx in range(len(test_data)):
        entry = test_data[idx]
        if entry['shape_id'] == shape_id:
            return entry, 'test', idx

    return None, None, None

# Search for your specific shape
target_shape_id = "f649133ee152f0c4535dab46efb28e27"
shape_entry, split_found, idx = find_shape_in_all_splits(target_shape_id, train_data, val_data, test_data)

if shape_entry is not None:
    print(f"Found shape {target_shape_id} in {split_found} split at index {idx}")
    print(f"Shape class: {shape_entry['shape_class']}")
    print(f"Affordances: {shape_entry['affordances']}")
    print(f"Point cloud shape: {shape_entry['coords'].shape}")
    print("\nFull shape entry:")
    for key, value in shape_entry.items():
        if isinstance(value, torch.Tensor):
            print(f"{key}: Tensor of shape {value.shape}")
        else:
            print(f"{key}: {value}")
else:
    # If not found, let's check the original dataset before splitting
    for idx in range(len(dataset)):
        entry = dataset[idx]
        if entry['shape_id'] == target_shape_id:
            print(f"Found shape in original dataset at index {idx}")
            print(f"Shape class: {entry['shape_class']}")
            print(f"Affordances: {entry['affordances']}")
            print(f"Point cloud shape: {entry['coords'].shape}")
            break
    else:
        print(f"Shape {target_shape_id} not found in any split or the original dataset!")

In [ ]:
def compute_highlight_iou(shape_entry, highlighted_npz_path, threshold=0.0):
    """
    Compare ground truth labels with highlighted predictions and compute IoU

    Args:
        shape_entry: The dataset entry containing ground truth labels
        highlighted_npz_path: Path to the NPZ file with highlighted predictions
        threshold: Threshold for converting prediction probabilities to binary

    Returns:
        dict: Dictionary containing IoU and related metrics
    """
    import numpy as np

    # Get ground truth labels
    gt_labels = shape_entry['labels_dict']['contain'].cpu().numpy()
    gt_binary = (gt_labels > 0).astype(int)

    # Load highlighted predictions
    pred_data = np.load('/content/highlighted_points.npz')
    pred_points = pred_data['points']         # shape (N, 3)
    pred_probs = pred_data['probabilities']   # shape (N, 2) typically

    # Get the highlight probabilities (assuming first column is highlight prob)
    highlight_probs = pred_probs[:, 0]
    pred_binary = (highlight_probs >= threshold).astype(int)

    # Verify shapes match
    coords = shape_entry['coords'].cpu().numpy()
    if coords.shape == pred_points.shape:
        print("Points match exactly in shape!")
        # Direct comparison
        intersection = np.sum((pred_binary == 1) & (gt_binary == 1))
        union = np.sum((pred_binary == 1) | (gt_binary == 1))
        iou = intersection / union if union > 0 else 0.0

    # else:
    #     print("Points don't match exactly, using nearest neighbor matching...")
    #     # Use KD-tree for point matching
    #     from scipy.spatial import cKDTree
    #     tree = cKDTree(coords)
    #     _, nn_indices = tree.query(pred_points)

    #     # Align ground truth to prediction ordering
    #     aligned_gt = gt_binary[nn_indices]

    #     # Compute IoU
    #     intersection = np.sum((pred_binary == 1) & (aligned_gt == 1))
    #     union = np.sum((pred_binary == 1) | (aligned_gt == 1))
    #     iou = intersection / union if union > 0 else 0.0

    # Compute additional metrics
    metrics = {
        'iou': iou,
        'intersection': intersection,
        'union': union,
        'gt_positive': np.sum(gt_binary),
        'pred_positive': np.sum(pred_binary),
        'total_points': len(gt_binary)
    }

    return metrics

# Usage example:
highlighted_npz_path = "path/to/your/highlighted_points.npz"  # Replace with your NPZ file path
metrics = compute_highlight_iou(shape_entry, highlighted_npz_path)

# Print results
print("\nResults:")
print(f"IoU Score: {metrics['iou']:.4f}")
print(f"Intersection: {metrics['intersection']} points")
print(f"Union: {metrics['union']} points")
print(f"Ground Truth Positive Points: {metrics['gt_positive']}")
print(f"Predicted Positive Points: {metrics['pred_positive']}")
print(f"Total Points: {metrics['total_points']}")


In [ ]:
def compute_average_iou(shape_entry, highlighted_npz_path, threshold_start=0.001, threshold_end=0.99, threshold_step=0.01):
    """
    Compute average IoU across different thresholds

    Args:
        shape_entry: Dataset entry containing ground truth labels
        highlighted_npz_path: Path to NPZ file with predictions
        threshold_start: Starting threshold value (default: 0.0)
        threshold_end: Ending threshold value (default: 0.99)
        threshold_step: Step size for threshold increments (default: 0.01)

    Returns:
        dict: Dictionary containing aIoU and IoUs at each threshold
    """
    import numpy as np
    from scipy.spatial import cKDTree

    # Get ground truth labels
    gt_labels = shape_entry['labels_dict']['openable'].cpu().numpy()
    gt_binary = (gt_labels > 0).astype(int)
    coords = shape_entry['coords'].cpu().numpy()

    # Load highlighted predictions
    pred_data = np.load(highlighted_npz_path)
    pred_points = pred_data['points']
    pred_probs = pred_data['probabilities']
    highlight_probs = pred_probs[:, 0]  # Assuming first column is highlight probability

    # Check if points need alignment
    if coords.shape != pred_points.shape:
        print("Aligning points using nearest neighbors...")
        tree = cKDTree(coords)
        _, nn_indices = tree.query(pred_points)
        gt_binary = gt_binary[nn_indices]

    # Generate thresholds
    thresholds = np.arange(threshold_start, threshold_end + threshold_step, threshold_step)
    ious = []

    print("Computing IoUs for different thresholds...")
    for threshold in thresholds:
        # Binarize predictions at this threshold
        pred_binary = (highlight_probs >= threshold).astype(int)

        # Compute IoU
        intersection = np.sum((pred_binary == 1) & (gt_binary == 1))
        union = np.sum((pred_binary == 1) | (gt_binary == 1))
        iou = intersection / union if union > 0 else 0.0
        ious.append(iou)

    # Convert to numpy array for easier manipulation
    ious = np.array(ious)

    # Compute average IoU
    aiou = np.mean(ious)

    # Find best threshold and its IoU
    best_idx = np.argmax(ious)
    best_threshold = thresholds[best_idx]
    best_iou = ious[best_idx]

    results = {
        'aiou': aiou,
        'thresholds': thresholds,
        'ious': ious,
        'best_threshold': best_threshold,
        'best_iou': best_iou
    }

    # Print summary
    print("\nResults:")
    print(f"Average IoU (aIoU): {aiou:.4f}")
    print(f"Best IoU: {best_iou:.4f} at threshold {best_threshold:.2f}")

    return results

# Optional: Visualize the IoU curve
def plot_iou_curve(results):
    import matplotlib.pyplot as plt

    plt.figure(figsize=(10, 6))
    plt.plot(results['thresholds'], results['ious'], 'b-', label='IoU')
    plt.axhline(y=results['aiou'], color='r', linestyle='--', label=f'aIoU = {results["aiou"]:.4f}')
    plt.plot(results['best_threshold'], results['best_iou'], 'go',
             label=f'Best IoU = {results["best_iou"]:.4f} at {results["best_threshold"]:.2f}')

    plt.xlabel('Threshold')
    plt.ylabel('IoU')
    plt.title('IoU vs Threshold')
    plt.grid(True)
    plt.legend()
    plt.show()

# Usage example:
highlighted_npz_path = "/content/highlighted_points.npz"  # Replace with your NPZ file path
results = compute_average_iou(shape_entry, highlighted_npz_path)

# Plot the results
plot_iou_curve(results)